In [1]:
import numpy as np
import math
import pickle
from collections import defaultdict
import re

import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet

import pyaspeller
from pymystem3 import Mystem
from langdetect import detect

from string import punctuation
from PyDictionary import PyDictionary
from rank_bm25 import BM25Plus
from gensim.summarization.bm25 import get_bm25_weights

import googletrans
#from translate import Translator
from yandex.Translater import Translater

import sys
import time
import os
import platform
from itertools import combinations
import multiprocessing
from io import StringIO
from multiprocessing import freeze_support

from lxml import etree
from html.parser import HTMLParser


[nltk_data] Downloading package wordnet to /home/twinkle/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
N_DOCUMENTS = 38114
MEAN_DOCLEN = 12155
TOTAL_LEMMS = N_DOCUMENTS * MEAN_DOCLEN

K1 = 1
K2 = 1 / 800

W_PAIR = 0.3

W_P = 2
W_I = 0.7
W_S = 0.5
W_W = 0.3

W_ALL = 0.2


In [3]:
queries_path = './data/queries.numerate.txt'
queries_processed_path = './data/queries.numerate_processed_stem.txt'

change_lang = ('da', 'sq', 'cy')
change_rule = {
    'q': 'й', 'w': 'ц', 'e': 'у', 'r': 'к', 't': 'е', 'y': 'н', 'u': 'г', 'i': 'ш',
    'o': 'щ', 'p': 'з', '[': 'х', ']': 'ъ', 'a': 'ф', 's': 'ы', 'd': 'в', 'f': 'а',
    'g': 'п', 'h': 'р', 'j': 'о', 'k': 'л', 'l': 'д', ';': 'ж', '\'': 'э', 'z': 'я',
    'x': 'ч', 'c': 'с', 'v': 'м', 'b': 'и', 'n': 'т', 'm': 'ь', ',': 'б', '.': 'ю'
    }

digits = set('1234567890')
en_symbols = set('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ')
ru_symbols = set('йцукенгшщзхъэждлорпавыфячсмитьбюЙЦУКЕНГШЩЗХЪЭЖДЛОРПАВЫФЯЧСМИТЬБЮ')

stemmer = Mystem()
tr = Translater()
tr.set_key('trnsl.1.1.20200117T155615Z.126b37a61e400bbc.afcd42f0d95690d2cbe61991736209c6a6214259')

speller = pyaspeller.YandexSpeller(find_repeat_words=True, ignore_capitalization=True)


In [4]:
def process_query(query):
    if len(query) == 0:
        return query

    if detect(query) in change_lang:
        query = ''.join(change_rule[c] if c in change_rule.keys() else c for c in query)

    query = ''.join([c if c in en_symbols or c in ru_symbols or c in digits else ' ' for c in query])
    words = [word.lower().strip() for word in query.split(' ')]
    query = ' '.join(words)
    lemmas = stemmer.lemmatize(query)
    query = ' '.join(word for word in lemmas if len(word) > 0 and word != '\n')
    query = re.sub(' +', ' ', query)
    return query

with open(queries_path, 'r', encoding='UTF-8') as queries_file, \
        open(queries_processed_path, 'w', encoding='UTF-8') as queries_processed_file:

    for line in queries_file:
        idx, query = (part.strip() for part in line.split('\t'))
        query = process_query(query)
        correction = speller.spell(query) 

        if len(list(correction)) > 0:
            query_ex = query
            any_corrections = False
            for w in speller.spell(query):
                if len(w['s']) > 0:
                    any_corrections = True
                    query = query.replace(w['word'], w['s'][0])
            query_main = process_query(query)
            if any_corrections == False:
                query_ex = ''
        else:
            query_ex = ''
            query_main = query

        tr.set_text(query_main)
        tr.set_from_lang(tr.detect_lang())
        tr.set_to_lang('en')
        trans = tr.set_text(tr.translate())
        tr.set_from_lang('en')
        tr.set_to_lang('ru')
        query_trans_main = process_query(tr.translate())

        if query_ex:
            tr.set_text(query_ex)
            tr.set_from_lang(tr.detect_lang())
            tr.set_to_lang('en')
            trans = tr.set_text(tr.translate())
            tr.set_from_lang('en')
            tr.set_to_lang('ru')
            query_trans_ex = process_query(tr.translate())
        else:
            query_trans_ex = ''

        print(idx)
        print(query_main)
        print(query_ex)
        print(query_trans_main)
        print(query_trans_ex)
        print('---- ---- ----')

        queries_processed_file.write(idx + '\t' + query_main.strip() + '\t' + \
                                     query_ex.strip() + '\t' + query_trans_main.strip() + '\t' + query_trans_ex.strip() + '\n')

1
мультивиз в израиль какой страна можно посещать

мультивиз в израиль как страна можно посещать

---- ---- ----
2
надо ли носить компресс гольф после операция на голеностоп
надо ли носить компрес гольф после операция на голеностоп
нужно ли носить компресс в гольф после операция на лодыжка
нужно ли я носить оборачивать гольф после операция на лодыжка
---- ---- ----
3
жировик на спина можно ли применять пиявка

шишка на спина можно ли применять пиявка

---- ---- ----
4
как прописывать просто админ
как прописывать просто адмика
как зарегистрировать администратор
как зарегистрировать админ
---- ---- ----
5
хто буде судить суперкубок 2017 м ж шахтер динамо кий в дбудеться в одес 

хто буде судити суперкубок 2017 м ж шахтар динамо якия в дбудеться в одес 

кто быть судить суперкубок 2017 шахтер динамо кий в budelse в одесса
кто быть судить суперкубок 2017 шахтер динамо акиа в budelse в одесса
---- ---- ----
6
как доезжать от метро мякинино до крокус сити молла

как добираться от станция мет

73
как избавляться от муравить в на огород народный средство

как избавляться от мораль в сад народный средство

---- ---- ----
74
что такой асс
что такой асц
это жопа
такой асц
---- ---- ----
75
как вставлять рисунок из бронза в железо

как вставлять картинка из бронза к железо

---- ---- ----
76
какой растение сажать в пруд

какой растение посадить в пруд

---- ---- ----
77
как понимать что ты нравиться парень

как понимать что ты нравиться парень

---- ---- ----
78
как закрывать доступ к страница mail ru

как закрывать доступ к почта страница ru

---- ---- ----
79
кто рождаться сегодня

кто рождаться сегодня

---- ---- ----
80
как добавлять админ в группа

как добавлять администратор в группа

---- ---- ----
81
как расширять экран гта 5

как расширять экран гта 5

---- ---- ----
82
как крепить стропило

как прикреплять стропило

---- ---- ----
83
почему черный икра терять цвет

почему черный икра потерять цвет

---- ---- ----
84
хороший средство от колорадский жук
хороший средство о

148
как ия льгота предусматривать закон по участник это принимать совмин кнр читать
как ия льгота предусматривать закон по участник ато принимать совмин лнр читать
как иа благо предоставлять право участник принимать совет министр кнр читать
как иа благо предоставлять право на ато чтобы взять совет министр лнр читать
---- ---- ----
149
поменять право в связь с истечение срок годность что нужно 2017

менять право в связь с истечение срок действие для что нужный 2017

---- ---- ----
150
как копировать контакт с симка в нокиа люмен 520
как копировать контакт с смки в нокиа люмиа 520
как скопировать контакт с sim карта в nokia 520 люмен
как скопировать контакт с smci в nokia lumia 520
---- ---- ----
151
где продаваться собака в кызылорд

где продавать собака в кызылорд

---- ---- ----
152
самый крутой мерседес

самый крутой мерседес

---- ---- ----
153
почему у хоста желтеть лист

почему желтеть лист хост

---- ---- ----
154
чем быть заполнять статуя колосс родосский для больший устойчивост

224
какой бывать вагонка на потолок

что происходить вагонка на потолок

---- ---- ----
225
как сделать детализация звонок на теле2 чужой номер бесплатно казахстан

как сделать детализация вызов на чужой номер теле2 бесплатно казахстан

---- ---- ----
226
как открывать весь персонаж в don t starve shipwrecked

как открывать весь персонаж в don т подыхать кораблекрушение

---- ---- ----
227
как альхам текст

как alham текст

---- ---- ----
228
что если чесаться правый глаз

если чесаться правый глаз

---- ---- ----
229
чем замазывать отверстие в евровагонка

чем покрывать отверстие в подкладка

---- ---- ----
230
как обшивать фундамент профлист

как обшивать фундамент металлический лист

---- ---- ----
231
как самостоятельно заливать фундамент для дом с мансарда

как заливать фундамент для дом с мансарда

---- ---- ----
232
чем замазывать шов между плитка в ванная чтобы не стираться

чем прикрывать шов между плитка в ванная так как не исчезать

---- ---- ----
233
почему в днепр выдавлив

299
как заменять поплавок в унитаз с каков подводка
как заменять поплавок в унитаз с окова подводка
как заменять поплавок в унитаз что такой подводка
как заменять поплавок в унитаз с кандалы подводка
---- ---- ----
300
сколько по время идти крещение ребенок

сколько время ехать на крещение ребенок

---- ---- ----
301
как убирать косточка с нога

как убирать косточка с нога

---- ---- ----
302
как правильно выбирать лимон

как выбирать правильный лимон

---- ---- ----
303
сколько человек погибать в война и преступность

сколько человек погибать в война и преступность

---- ---- ----
304
как находить кнопка термовыключатель в водонагреватель термекс

как находить кнопка тепловой реле в водонагреватель термекс

---- ---- ----
305
где скачать карта на майн крафт на 1 7 10
где скачать карта на майнкрафт на 1 7 10
где скачать карта на майнкрафт 1 7 10
где скачать карта для майнкрафт 1 7 10
---- ---- ----
306
какой время в америка

сколько время в америка

---- ---- ----
307
какой детский кре

367
как изменять имя персонаж в игра сталкер

как изменять имя персонаж в игра

---- ---- ----
368
что приготавливать из старый молочка

приготавливать кислый молоко

---- ---- ----
369
как правильно наносить жидкий стекло wilson
как правильно наносить жидкий стекло willson
как применять жидкий стекло вилсон
как применять жидкий стекло уиллсон
---- ---- ----
370
чем отличаться зубной врач от стоматолог

отличаться стоматолог от дантист

---- ---- ----
371
как оформлять загранпаспорт

как оформлять загранпаспорт

---- ---- ----
372
как посмотреть сколько оставаться трафик на мтс

как посмотреть сколько оставаться трафик на мтс

---- ---- ----
373
как перенести убыток прошлый год если налоговый учет не вестись

как перенести убыток прошлый год если налоговый учет не быть

---- ---- ----
374
сколько калория в дыня колхозница

сколько калория в дыня фермер

---- ---- ----
375
хороший фильм с джеки чан смотреть онлайн

хороший фильм с джеки чан смотреть онлайн

---- ---- ----
376
что пригот

In [5]:
def get_text_stem_files(base_path):
    files = []
    text_dir_paths = [f for f in os.listdir(base_path) if f.count('_') == 3 and f.find('text_stem') >= 0]
    for dir_path in text_dir_paths:
        for file_path in os.listdir(base_path + dir_path):
            yield base_path + dir_path + '/' + file_path


def get_title_stem_files(base_path):
    files = []
    text_dir_paths = [f for f in os.listdir(base_path) if f.count('_') == 3 and f.find('title_stem') >= 0]
    for dir_path in text_dir_paths:
        for file_path in os.listdir(base_path + dir_path):
            yield base_path + dir_path + '/' + file_path


def get_base_file_path(path):
    if path.find('text'):
        path = path.replace('_text', '')
    if path.find('title'):
        path = path.replace('_title', '')
    if path.find('stem'):
        path = path.replace('_stem', '')
    return path


def file_path_to_tf_path(path):
    return path.replace('content', 'statistics/tf').replace('.dat', '.pkz')


def file_path_to_pairs_path(path):
    return path.replace('content', 'statistics/pairs').replace('.dat', '.pkz')


def file_path_to_hop_pairs_path(path):
    return path.replace('content', 'statistics/hop_pairs').replace('.dat', '.pkz')

def file_path_to_hop_hop_pairs_path(path):
    return path.replace('content', 'statistics/hop_hop_pairs').replace('.dat', '.pkz')


In [6]:
base_path = './data/content/'
url_path = './data/urls.numerate.txt'
num_path = './data/path.numerate.txt'
num_sort_path = './data/path.numerate.sort.txt'
p_n_path = './data/path.numerate.pkz'
n_p_path = './data/numerate.path.pkz'

path_numerate = dict()
numerate_path = dict()
url_to_num = dict()

dir_paths = [f for f in os.listdir(base_path) if f.count('_') == 1]

with open(url_path, 'r', encoding='UTF-8', errors='ignore') as url_file:
    for line in url_file:
        num, url = line.strip().split('\t')
        url_to_num[url] = num

cnt = 0
with open(num_path, 'a', encoding='UTF-8', errors='ignore') as num_file:
    for dir_path in dir_paths:
        file_paths = [f for f in os.listdir(base_path + dir_path)]
        for file_path in file_paths:
            full_file_path = base_path + dir_path + '/' + file_path
            with open(full_file_path, 'r', encoding='UTF-8', errors='ignore') as file:
                url = file.readline().strip()
                num_file.write(full_file_path + '\t' + str(url_to_num[url]) + '\n')
                print('\r{}'.format(cnt), end='')
                cnt += 1

cnt = 0
with open(num_path, 'r', encoding='UTF-8', errors='ignore') as num_file, \
    open(num_sort_path, 'w', encoding='UTF-8', errors='ignore') as num_sort_file:
    paths = defaultdict(str)
    for line in num_file:
        path, idx = (part.strip() for part in line.split('\t'))
        paths[idx] = path
        print('\r{}'.format(cnt), end='')
        cnt += 1
    for i in range(1, int(N_DOCUMENTS) + 1):
        num_sort_file.write(paths[str(i)] + '\t' + str(i) + '\n')

print()        

cnt = 0
with open(num_path, 'r', encoding='UTF-8', errors='ignore') as num_file, \
        open(p_n_path, 'wb') as p_n_file, open(n_p_path, 'wb') as n_p_file:
    for line in num_file:
        path, num = line.strip().split('\t')
        path_numerate[path] = int(num)
        numerate_path[int(num)] = path
        print('\r{}'.format(cnt), end='')
        cnt += 1
    pickle.dump(path_numerate, p_n_file)
    pickle.dump(numerate_path, n_p_file)

38113
38113

In [7]:
queries_path = './data/queries.numerate_processed_stem.txt'
base_path = './data/queries.numerate_processed_chunks/'
n_splits = 4
split_size = 100

split_files = [open(base_path + 'chunk_{}'.format(str(idx)) + '.txt',
                    'w',
                    encoding='UTF-8',
                    errors='ignore')
               for idx in range(1, n_splits + 1)]

with open(queries_path, 'r', encoding='UTF-8', errors='ignore') as queries_file:
    for line in queries_file:
        idx, query_main, query_ex, query_trans_main, query_trans_ex = (part.strip() for part in line.split('\t'))
        split_files[int(idx) // split_size].write(idx + '\t' + query_main.strip() + '\t' + query_ex.strip() + '\t' + query_trans_main.strip() + '\t' + query_trans_ex.strip() + '\n')

In [8]:
class TextHTMLParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self._text = []
        self._title = []
        self._in_title = False

    def handle_data(self, text):
        text = text.strip()
        if len(text) > 0:
            if self.accept_field(text) or self._in_title:
                pass
            else:
                return
            if self._in_title:
                self._title.append(text)
            else:
                self._text.append(text)

    def handle_starttag(self, tag, attrs):
        if tag == 'title':
            self._in_title = True

    def handle_endtag(self, tag):
        if tag == 'title':
            self._in_title = False

    def text(self):
        text = ' '.join(self._text)
        # only literals or digits
        text = re.sub('\W+', ' ', text)
        # split number from text, make lower
        text = ' '.join(re.split('(\d+)', text)).lower()
        text = ' '.join(text.split(' '))
        return text

    def title(self):
        text = ' '.join(self._title)
        # only literals or digits
        text = re.sub('\W+', ' ', text)
        # split number from text, make lower
        text = ' '.join(re.split('(\d+)', text)).lower()
        text = ' '.join(text.split(' '))
        return text

    def accept_field(self, field):
        en_cnt = 0
        ru_cnt = 0
        di_cnt = 0
        for c in field:
            if c in en_symbols:
                en_cnt += 1
            elif c in ru_symbols:
                ru_cnt += 1
            elif c in digits:
                di_cnt += 1
        if en_cnt + ru_cnt == 0:
            return False
        en_proba = en_cnt / (en_cnt + ru_cnt)
        if en_proba < 0.95 or en_cnt == len(field) - ru_cnt - field.count(' '):
            return True
        return False


In [9]:
def get_text_and_title(path, *args):
    with open(path, 'r', encoding='UTF-8', errors='ignore') as file:
        text = file.read()

        # fix DOM tree using lxml tools
        parser = etree.HTMLParser()
        tree = etree.parse(StringIO(text), parser)
        text = str(etree.tostring(tree.getroot(), pretty_print=True, method="html"), encoding='UTF-8')

        # parse fixed html and get title, text, meta, headers, ...
        parser = TextHTMLParser()
        parser.feed(text)
        return parser.text(), parser.title()

def get_text_and_title_stem(text_path, title_path):
    with open(text_path, 'r', encoding='UTF-8', errors='ignore') as text_file, \
            open(title_path, 'r', encoding='UTF-8', errors='ignore') as title_file:
        text = text_file.read()
        title = title_file.read()

    text_words, title_words = stemmer.lemmatize(text), stemmer.lemmatize(title)

    text = ' '.join(word for word in text_words if len(word) > 0 and word != '\n')
    title = ' '.join(word for word in title_words if len(word) > 0 and word != '\n')
    text = ' '.join(text.split(' '))
    title = ' '.join(title.split(' '))
    return text, title


def prepare_dir(base_path, step_suffix, dir_path, preparer):
    cnt = 0
    # creating directories
    try:
        os.mkdir(base_path + dir_path + step_suffix[0])
    except FileExistsError:
        pass
    try:
        os.mkdir(base_path + dir_path.replace('text', 'title') + step_suffix[1])
    except FileExistsError:
        pass

    file_paths = [f for f in os.listdir(base_path + dir_path)]
    for file_path in file_paths:
        real_path = base_path + dir_path + '/' + file_path
        text_real_path_result = base_path + dir_path + step_suffix[0] + '/' + file_path
        title_real_path_result = base_path + dir_path.replace('text', 'title') + step_suffix[1] + '/' + file_path

        if os.path.exists(text_real_path_result) and os.path.exists(title_real_path_result):
            continue

        text, title = preparer(real_path, real_path.replace('text', 'title'))

        with open(text_real_path_result, 'w', encoding='UTF-8', errors='ignore') as result_file:
            result_file.write(text)
        with open(title_real_path_result, 'w', encoding='UTF-8', errors='ignore') as result_file:
            result_file.write(title)
        print('\r{}'.format(cnt), end='')
        cnt += 1


In [10]:
base_path = './data/content/'
dir_paths = [f for f in os.listdir(base_path) if f.count('_') == 1]
step_suffix = ['_text', '_title']

for dir_path in dir_paths:
    # get text and title for each document
    prepare_dir(base_path, step_suffix, dir_path, get_text_and_title)

38113


In [11]:
base_path = './data/content/'
dir_paths = [f for f in os.listdir(base_path) if f.count('_') == 2 and f.find('text') >= 0]
step_suffix = ['_stem', '_stem']

for dir_path in dir_paths:
    # get stemmed text and title fot each document
    prepare_dir(base_path, step_suffix, dir_path, get_text_and_title_stem)

38113


In [12]:
def count_tf(file_generator):
    cnt = 0
    for file_path in file_generator:
        print('\r{}'.format(cnt), end='')
        cnt += 1
        tf_dict = defaultdict(int)
        with open(file_path, 'r', encoding='UTF-8', errors='ignore') as file:
            text = file.read().strip()
            words = text.split(' ')
            for word in words:
                tf_dict[word] += 1

        tf_path = file_path_to_tf_path(file_path)
        os.makedirs(os.path.dirname(tf_path), exist_ok=True)
        with open(tf_path, 'wb') as tf_file:
            pickle.dump(tf_dict, tf_file)

base_path = './data/content/'
base_result_path = './data/statistics/'

count_tf(get_text_stem_files(base_path))
count_tf(get_title_stem_files(base_path))


38113

In [13]:
def count_df(file_generator, df_path):
    df_dict = defaultdict(int)
    cnt = 0
    for file_path in file_generator:
        print('\r{}'.format(cnt), end='')
        cnt += 1
        tmp_dict = defaultdict(int)
        with open(file_path, 'r', encoding='UTF-8', errors='ignore') as file:
            text = file.read().strip()
            words = text.split(' ')
            for word in words:
                tmp_dict[word] = 1
        for key in tmp_dict.keys():
            df_dict[key] += 1
    with open(df_path, 'wb') as df_file:
        pickle.dump(df_dict, df_file)


base_path = './data/content/'
base_result_path = './data/statistics/'

text_df_path = base_result_path + 'text_df_count.pkz'
title_df_path = base_result_path + 'title_df_count.pkz'

count_df(get_text_stem_files(base_path), text_df_path)
count_df(get_title_stem_files(base_path), title_df_path)


38113

In [14]:
def count_cf(file_generator, cf_path):
    cf_dict = defaultdict(int)
    cnt = 0
    for file_path in file_generator:
        print('\r{}'.format(cnt), end='')
        cnt += 1
        with open(file_path, 'r', encoding='UTF-8', errors='ignore') as file:
            text = file.read().strip()
            words = text.split(' ')
            for word in words:
                cf_dict[word] += 1
    with open(cf_path, 'wb') as cf_file:
        pickle.dump(cf_dict, cf_file)        
        
base_path = './data/content/'
base_result_path = './data/statistics/'

text_cf_path = base_result_path + 'text_cf_count.pkz'
title_cf_path = base_result_path + 'title_cf_count.pkz'

count_cf(get_text_stem_files(base_path), text_cf_path)
count_cf(get_title_stem_files(base_path), title_cf_path)


38113

In [15]:
base_path = './data/content/'
p_n_path = './data/path.numerate.pkz'
text_doclen_path = './data/statistics/text_doclen.pkz'
title_doclen_path = './data/statistics/title_doclen.pkz'

with open(p_n_path, 'rb') as p_n_file:
    path_to_num = pickle.load(p_n_file)
    num_to_text_len = defaultdict(int)
    num_to_title_len = defaultdict(int)

    cnt = 0
    for path in get_text_stem_files(base_path):
        print('\r{}'.format(cnt), end='')
        cnt += 1
        base = get_base_file_path(path)
        with open(path, 'r', encoding='UTF-8', errors='ignore') as file:
            num_to_text_len[int(path_to_num[base])] = len(file.read().strip().split(' '))

    with open(text_doclen_path, 'wb') as text_doclen_file:
        pickle.dump(num_to_text_len, text_doclen_file)

    print()

    cnt = 0
    for path in get_title_stem_files(base_path):
        print('\r{}'.format(cnt), end='')
        cnt += 1
        base = get_base_file_path(path)
        with open(path, 'r', encoding='UTF-8', errors='ignore') as file:
            num_to_title_len[int(path_to_num[base])] = len(file.read().strip().split(' '))

    with open(title_doclen_path, 'wb') as title_doclen_file:
        pickle.dump(num_to_title_len, title_doclen_file)


38113
38113

In [16]:
def count_pairs(file_generator):
    cnt = 0
    for file_path in file_generator:
        print('\r{}'.format(cnt), end='')
        cnt += 1
        pairs_dict = defaultdict(int)
        with open(file_path, 'r', encoding='UTF-8', errors='ignore') as file:
            text = file.read().strip()
            words = text.split(' ')
            for idx in range(len(words) - 1):
                pairs_dict[(words[idx], words[idx + 1])] += 1

        pairs_path = file_path_to_pairs_path(file_path)
        os.makedirs(os.path.dirname(pairs_path), exist_ok=True)
        with open(pairs_path, 'wb') as pairs_file:
            pickle.dump(pairs_dict, pairs_file)


base_path = './data/content/'
base_result_path = './data/statistics/'

count_pairs(get_text_stem_files(base_path))
count_pairs(get_title_stem_files(base_path))


38113


In [17]:
def count_hop_pairs(file_generator):
    cnt = 0
    for file_path in file_generator:
        print('\r{}'.format(cnt), end='')
        cnt += 1
        hop_pairs_dict = defaultdict(int)
        with open(file_path, 'r', encoding='UTF-8', errors='ignore') as file:
            text = file.read().strip()
            words = text.split(' ')
            for idx in range(len(words) - 2):
                hop_pairs_dict[(words[idx], words[idx + 2])] += 1

        hop_pairs_path = file_path_to_hop_pairs_path(file_path)
        os.makedirs(os.path.dirname(hop_pairs_path), exist_ok=True)
        with open(hop_pairs_path, 'wb') as pairs_file:
            pickle.dump(hop_pairs_dict, pairs_file)


base_path = './data/content/'
base_result_path = './data/statistics/'

count_hop_pairs(get_text_stem_files(base_path))
count_hop_pairs(get_title_stem_files(base_path))


38113


In [18]:
def count_hop_hop_pairs(file_generator):
    cnt = 0
    for file_path in file_generator:
        print('\r{}'.format(cnt), end='')
        cnt += 1
        hop_hop_pairs_dict = defaultdict(int)
        with open(file_path, 'r', encoding='UTF-8', errors='ignore') as file:
            text = file.read().strip()
            words = text.split(' ')
            for idx in range(len(words) - 3):
                hop_hop_pairs_dict[(words[idx], words[idx + 3])] += 1

        hop_hop_pairs_path = file_path_to_hop_hop_pairs_path(file_path)
        os.makedirs(os.path.dirname(hop_hop_pairs_path), exist_ok=True)
        with open(hop_hop_pairs_path, 'wb') as pairs_file:
            pickle.dump(hop_hop_pairs_dict, pairs_file)


base_path = './data/content/'
base_result_path = './data/statistics/'

count_hop_hop_pairs(get_text_stem_files(base_path))
count_hop_hop_pairs(get_title_stem_files(base_path))


38113

In [19]:
# attempt to implement an assessor evaluation
# (the result hasn't been improved)

sample_path = './data/sample.technosphere.ir1.textrelevance.submission.txt'
base_path = './data/path.numerate.sort.txt'
base_result_path = './data/statistics/b25.similarity.pkz'
N = 400

def query_to_doc_order(base):
    current_id = 1
    min = 1
    max = 1
    query_docs = defaultdict(list)
    docs_query = defaultdict(list)

    for i in range(1, N):
        query_docs[i] = [[], []]

    with open(base, 'r', encoding='UTF-8') as file:
        file.readline()
        for line in file:
            query_id, doc_id = map(int, (part.strip() for part in line.split(',')))
            docs_query[doc_id].append(query_id)
            if query_id == current_id:
                if min <= doc_id <= max:
                    max += 1
                else:
                    query_docs[query_id][1].append(doc_id)
            else:
                query_docs[current_id][0] = [min, max - 1]
                current_id = query_id
                min = max
                if min <= doc_id <= max:
                    max += 1
                else:
                    query_docs[query_id][1].append(doc_id)
        query_docs[query_id][0] = [min, max - 1]
    return query_docs, docs_query

def path_to_text(base, docs):
    texts = list()
    with open(base, 'r', encoding='UTF-8') as file:
        for line in file:
            path, doc_id = (part.strip() for part in line.split('\t'))
            doc_id = int(doc_id)
            if doc_id in docs:
                tokens = path.split('/')
                text_path = path.replace(tokens[3], tokens[3] + '_text_stem')
                title_path = path.replace(tokens[3], tokens[3] + '_title_stem')
                with open(text_path, 'r', encoding='UTF-8') as text, \
                    open(title_path, 'r', encoding='UTF-8') as title:
                    t1 = title.read().strip()
                    t2 = text.read().strip()
                    texts.append((t1 + ' ' + t2).split())
    return texts

def py_bm25_sim(base, dest, sample):
    cnt = 0
    query_docs, docs_query = query_to_doc_order(sample)
    query_docs_sim = defaultdict(list)

    for i in range(1, N):
        cnt += 1
        print('\r{}'.format(cnt), end='')
        texts = path_to_text(base, list(range(query_docs[i][0][0], query_docs[i][0][1])) + query_docs[i][1])
        result = get_bm25_weights(texts, 4)
        query_docs_sim[i] = result

    with open(dest, 'wb', encoding='UTF-8') as docs_sim_file:
        pickle.dump(query_docs_sim, docs_sim_file)


In [20]:
def get_idf(word):
    cf = text_cf_dict[word] + title_cf_dict[word] + 1
    df = text_df_dict[word] + title_df_dict[word] + 1
    idf_1 = -math.log(1 - math.exp(-1.5 * cf / N_DOCUMENTS)) - math.log(df / N_DOCUMENTS)
    idf_2 = math.log(TOTAL_LEMMS / cf)
    return (idf_1 + idf_2) / 2


def get_score(query, doc_id, doc_base_path):
    if len(query) == 0:
        return 0

    text_tf_path = doc_base_path \
        .replace('content', 'statistics/tf').replace('/doc.', '_text_stem/doc.').replace('.dat', '.pkz')
    title_tf_path = doc_base_path \
        .replace('content', 'statistics/tf').replace('/doc.', '_title_stem/doc.').replace('.dat', '.pkz')

    text_pairs_path = doc_base_path \
        .replace('content', 'statistics/pairs').replace('/doc.', '_text_stem/doc.').replace('.dat', '.pkz')
    title_pairs_path = doc_base_path \
        .replace('content', 'statistics/pairs').replace('/doc.', '_title_stem/doc.').replace('.dat', '.pkz')

    text_hop_pairs_path = doc_base_path \
        .replace('content', 'statistics/hop_pairs').replace('/doc.', '_text_stem/doc.').replace('.dat', '.pkz')
    title_hop_pairs_path = doc_base_path \
        .replace('content', 'statistics/hop_pairs').replace('/doc.', '_title_stem/doc.').replace('.dat', '.pkz')

    text_hop_hop_pairs_path = doc_base_path \
        .replace('content', 'statistics/hop_hop_pairs').replace('/doc.', '_text_stem/doc.').replace('.dat', '.pkz')
    title_hop_hop_pairs_path = doc_base_path \
        .replace('content', 'statistics/hop_hop_pairs').replace('/doc.', '_title_stem/doc.').replace('.dat', '.pkz')

    score = 0
    with open(text_tf_path, 'rb') as text_tf_file, \
            open(title_tf_path, 'rb') as title_tf_file, \
            open(text_pairs_path, 'rb') as text_pairs_file, \
            open(title_pairs_path, 'rb') as title_pairs_file, \
            open(text_hop_pairs_path, 'rb') as text_hop_pairs_file, \
            open(title_hop_pairs_path, 'rb') as title_hop_pairs_file, \
            open(text_hop_hop_pairs_path, 'rb') as text_hop_hop_pairs_file, \
            open(title_hop_hop_pairs_path, 'rb') as title_hop_hop_pairs_file:

        text_tf_dict = pickle.load(text_tf_file)
        title_tf_dict = pickle.load(title_tf_file)

        text_pairs_dict = pickle.load(text_pairs_file)
        title_pairs_dict = pickle.load(title_pairs_file)

        text_hop_pairs_dict = pickle.load(text_hop_pairs_file)
        title_hop_pairs_dict = pickle.load(title_hop_pairs_file)

        text_hop_hop_pairs_dict = pickle.load(text_hop_hop_pairs_file)
        title_hop_hop_pairs_dict = pickle.load(title_hop_hop_pairs_file)

        words = query.split(' ')
        W_single = 0
        W_pair = 0
        W_all = 0

        doclen = float(text_doclen_dict[doc_id] + title_doclen_dict[doc_id])
        N_missed = 0
        sum_log_p = 0

        for word in words:
            tf = text_tf_dict[word] + title_tf_dict[word]
            hdr = title_tf_dict[word]

            IDF = get_idf(word)

            TF_1 = tf / (tf + K1 + K2 * doclen)
            TF_2 = 1 if hdr > 0 else 0

            W_single += IDF * (TF_1 + 1.5 * TF_2)

            sum_log_p += IDF
            if tf < 1:
                N_missed += 1

        for word_1, word_2 in combinations(words, 2):
            IDF_1 = get_idf(word_1)
            IDF_2 = get_idf(word_2)

            pair_1_2 = text_pairs_dict[(word_1, word_2)] + title_pairs_dict[(word_1, word_2)]
            pair_2_1 = text_pairs_dict[(word_2, word_1)] + title_pairs_dict[(word_2, word_1)]
            hop_pair_1_2 = text_hop_pairs_dict[(word_1, word_2)] + title_hop_pairs_dict[(word_1, word_2)]
            hop_hop_pair_1_2 = text_hop_hop_pairs_dict[(word_1, word_2)] + title_hop_hop_pairs_dict[(word_1, word_2)]

            TF = (W_P * pair_1_2 + W_S * hop_pair_1_2 + W_W * hop_hop_pair_1_2 + W_I * pair_2_1)
            W_pair += (IDF_1 + IDF_2) * TF / (1 + TF)

        W_all = sum_log_p * (0.03 ** N_missed)

        score = W_single + W_PAIR * W_pair + W_ALL * W_all

    return score


n_p_path = './data/numerate.path.pkz'

target_docs_path = './data/sample.technosphere.ir1.textrelevance.submission.txt'

text_doclen_path = './data/statistics/text_doclen.pkz'
title_doclen_path = './data/statistics/title_doclen.pkz'
text_cf_path = './data/statistics/text_cf_count.pkz'
title_cf_path = './data/statistics/title_cf_count.pkz'
text_df_path = './data/statistics/text_df_count.pkz'
title_df_path = './data/statistics/title_df_count.pkz'

with open(text_cf_path, 'rb') as text_cf_file, \
        open(title_cf_path, 'rb') as title_cf_file, \
        open(text_df_path, 'rb') as text_df_file, \
        open(title_df_path, 'rb') as title_df_file, \
        open(text_doclen_path, 'rb') as text_doclen_file, \
        open(title_doclen_path, 'rb') as title_doclen_file:

    text_cf_dict = pickle.load(text_cf_file)
    title_cf_dict = pickle.load(title_cf_file)
    text_df_dict = pickle.load(text_df_file)
    title_df_dict = pickle.load(title_df_file)
    text_doclen_dict = pickle.load(text_doclen_file)
    title_doclen_dict = pickle.load(title_doclen_file)


def process(part_idx):
    queries_path = './data/queries.numerate_processed_chunks/chunk_{0}.txt'.format(part_idx)
    submition_path = './data/queries.numerate_processed_chunks/results/chunk_{0}.txt'.format(part_idx)

    queries = defaultdict(tuple)
    with open(queries_path, 'r', encoding='UTF-8', errors='ignore') as file:
        for line in file:
            idx, query_main, query_ex, query_trans_main, query_trans_ex = (part.strip() for part in line.split('\t'))
            queries[int(idx)] = (query_main, query_ex, query_trans_main, query_trans_ex)

    with open(n_p_path, 'rb') as file:
        n_p_dict = pickle.load(file)

    result = defaultdict(list)
    queries_processed = 0
    checked = False
    with open(target_docs_path, 'r', encoding='UTF-8', errors='ignore') as file:
        file.readline()
        for line in file:
            query_id, doc_id = (int(val) for val in line.strip().split(','))
            if query_id not in queries:
                continue
            try:
                query_main, query_ex, query_trans_main, query_trans_ex = queries[query_id]
                score_main = get_score(query_main, doc_id, n_p_dict[doc_id])
                score_ex = get_score(query_ex, doc_id, n_p_dict[doc_id])
                score_trans_main = get_score(query_trans_main, doc_id, n_p_dict[doc_id])
                score_trans_ex = get_score(query_trans_ex, doc_id, n_p_dict[doc_id])
                score = max(score_main, score_ex, score_trans_main, score_trans_ex)
                if not checked:
                    print(part_idx,
                          doc_id,
                          np.round(score_main, 3),
                          np.round(score_ex, 3),
                          np.round(score_trans_main, 3),
                          np.round(score_trans_ex, 3))
                    checked = True
            except KeyError:
                score = 0
            result[query_id].append((doc_id, score))
            queries_processed += 1

    for key in result.keys():
        result[key] = sorted(result[key], key=lambda val: val[1], reverse=True)

    with open(submition_path, 'w') as file:
        file.write('QueryId,DocumentId\n')
        for key in sorted(result.keys()):
            for doc_id, _ in result[key]:
                file.write(str(key) + ',' + str(doc_id) + '\n')
    print(part_idx, 'done')


pool = multiprocessing.pool.ThreadPool(4)
pool.map(process, list(range(1, 5)))
pool.close()
pool.join()


1 1 16.599 0 16.93 0
4 28658 7.591 0 5.829 0
2 3 192159505 37.045 0 77.388 0
 13.244 0 10.629 0
1 done
4 done
2 done
3 done


In [21]:
base_path = './data/queries.numerate_processed_chunks/results/'
submition_path = base_path + 'out.submition_{0}.txt'.format(str(time.time()))

paths = sorted([f for f in os.listdir(base_path) if f.find('chunk') >= 0])

with open(submition_path, 'w') as submition_file:
    submition_file.write('QueryId,DocumentId\n')
    for path in paths:
        with open(base_path + path, 'r') as file:
            file.readline()
            for line in file:
                submition_file.write(line)
